In [ ]:
import os
from dotenv import load_dotenv
from crewai import Agent, LLM, Task, Crew, Process

# Load environment variables
load_dotenv()

# Option D: Azure OpenAI
openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
openai_model_name = os.getenv("AZURE_OPENAI_MODEL_NAME")

llm = LLM(
    model="azure/gpt-4o-mini",
    api_key=openai_api_key,
    base_url=openai_endpoint,
    api_version=openai_api_version,
    azure=True
)

print("🚀 Environment configured!")
print(f"LLM Model: {llm.model}")

## Hierarchical Process

In [2]:

# research a new project idea, do the research on market demand, risk, and potential return on investment.

# Define the Manager AI
manager_agent = Agent(
    role="Project Research Manager",
    goal="Oversee the project research and ensure timely, high-quality responses.",
    backstory="""An experienced project manager responsible for ensuring project research.
    When delegating work, provide simple string values for task, context, and coworker.
    Do NOT use dictionary formats - just provide plain text strings.""",
    allow_delegation=True,
    verbose=True,
    llm=llm
)

# Define the Technical Support AI
market_demand_agent = Agent(
    role="Market Demand Analyst",
    goal="Write market demand content.",
    backstory="""A skilled market demand analyst who
                 writes market demand content.""",
    allow_delegation=False, 
    verbose=True,
    llm=llm
)

# Define the Fiction Writer AI
risk_analysis_agent = Agent(
    role="Risk Analysis Analyst",
    goal="Write risk analysis content.",
    backstory="""A risk analysis analyst who
                 writes fiction content.""",
    allow_delegation=False, 
    verbose=True,
    llm=llm
)

# Define the Fiction Writer AI
return_on_investment_agent = Agent(
    role="Return on Investment Analyst",
    goal="Write return on investment content.",
    backstory="""A return on investment analyst who
                writes return on investment content.""",
    allow_delegation=False, 
    verbose=True,
    llm=llm
)


In [3]:
from crewai import Task

manager_task = Task(
    description="""Oversee the project research on {project_title} and ensure timely, high-quality responses.""",
    expected_output="A manager-approved response ready to be sent as an article on {project_title}.",
    agent=manager_agent, 
)

market_demand_task = Task(
    description="""Analyze the market demand for the project title '{project_title}'""",
    expected_output="A categorized project title labeled as 'Technical' or 'Fiction'.",
    agent=market_demand_agent, 
)

risk_analysis_task = Task(
    description="""Analyze the risk of the project title '{project_title}'""",
    expected_output="A categorized project title labeled as 'Technical' or 'Fiction'.",
    agent=risk_analysis_agent, 
)

return_on_investment_task = Task(
    description="""Analyze the return on investment of the project title '{project_title}'""",
    expected_output="A categorized project title labeled as 'Technical' or 'Fiction'.",
    agent=return_on_investment_agent, 
)

final_report_task = Task(
    description="""Review the final responses from the 
                   market demand, risk analysis, and return on investment agents
                   and create a final report.""",
    expected_output="""A comprehensive report on the project title '{project_title}'
    containing the market demand, risk analysis, and return on investment.""",
    agent=manager_agent,
)

In [4]:
from crewai import Crew, Process

project_research_crew = Crew(
    agents=[market_demand_agent, risk_analysis_agent, return_on_investment_agent],
    
    tasks=[market_demand_task, risk_analysis_task, return_on_investment_task, final_report_task],
    
    manager_agent=manager_agent,
    
    process=Process.hierarchical,
    
    verbose=True,
)

In [ ]:
inputs = {"project_title": "Multi-Agent System"}

result = project_research_crew.kickoff(inputs=inputs)

In [ ]:
from IPython.display import Markdown
Markdown(result.raw)